In [9]:
import json
import os
import pandas as pd
import random
import string
import time
import yaml

In [10]:
def char_string_gen(length):
    return ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(length))

def check_digit_gen(valid=True, ULI=None):
		"""Generates a check digit for a ULI in accordance with
		https://www.consumerfinance.gov/eregulations/diff/1003-C/2015-26607_20170101/2015-26607_20180101?from_version=2015-26607_20170101#1003-C-1"""
		if ULI is None:
			raise ValueError("a ULI must be supplied")
		#GENERATING A CHECK DIGIT
		#Step 1: Starting with the leftmost character in the string that consists of the combination of the
		#Legal Entity Identifier (LEI) pursuant to § 1003.4(a)(1)(i)(A) and the additional characters identifying the
		#covered loan or application pursuant to § 1003.4(a)(1)(i)(B), replace each alphabetic character with numbers
		#in accordance with Table I below to obtain all numeric values in the string.
		
		
		#1: convert letters to digits
		#2: append '00' to right of string
		#3:Apply the mathematical function mod=(n, 97) where n= the number obtained in step 2 above and 97 is the divisor.
		#3a: Alternatively, to calculate without using the modulus operator, divide the numbers in step 2 above by 97.
		#   Truncate the remainder to three digits and multiply it by .97. Round the result to the nearest whole number.
		#4: Subtract the result in step 3 from 98. If the result is one digit, add a leading 0 to make it two digits.
		#5: The two digits in the result from step 4 is the check digit. Append the resulting check digit to the
		#   rightmost position in the combined string of characters described in step 1 above to generate the ULI.
		
		#digit_vals contains the conversion of numbers to letters
		digit_vals = {
		'A':10, 'H':17,'O':24,'V':31,'B':11,'I':18,'P':25,'W':32,'C':12,'J':19,'Q':26,'X':33,'D':13,'K':20,'R':27,'Y':34,
		'E':14,'L':21,'S':28,'Z':35,'F':15,'M':22,'T':29,'G':16,'N':23,'U':30}
		
		uli_chars = list(ULI)
		mod_uli_chars = []
		for char in uli_chars:
			if char.upper() in digit_vals.keys():
				mod_uli_chars.append(str(digit_vals[char.upper()]))
			else:
				mod_uli_chars.append(char)
		mod_uli_chars.append('00') 
		digit_base = int("".join(mod_uli_chars))
		digit_modulo = digit_base % 97
		check_digit = 98 - digit_modulo

		if valid:
			return str(check_digit).zfill(2) #left pad check digit with 0 if length is less than 2
		else:
			return str(check_digit+6).zfill(2)[:2] #return a bad check digit (used in edit testing)

def uli_check(df=None):
    df['unique'] = df
    if len(df['unique'].unique()) == len(df.index):
        statement = "Done"
    else: 
        statement = "Rerun"
    print(statement)
    return df.drop('unique', axis = 1)
    print(df)


In [11]:
file = "edits_files/clean_file_1000_rows_Bank1.txt"

def large_files_bank1(file = None, count = None):
    """
    This function passes a clean 1000 row test file and a count that is a multiple of 1000, 
    to create large test files for bank 1 with the number of rows equal to count. 
    
    This function also uses the uli_check function to check for duplicate uli's. 
    """
    with open(file, 'r' ) as f:
        ts_row = f.readline()
        lar_rows = f.readlines()
    with open("edits_files/lar_rows.txt", 'w') as out_lar:
        out_lar.writelines(lar_rows)
    with open("edits_files/ts_row.txt", 'w') as out_ts:
        out_ts.writelines(ts_row)
    with open('schemas/lar_schema.json') as d:
        data = json.load(d)
        data = pd.DataFrame(data)
    
    df = pd.read_csv('edits_files/lar_rows.txt', delimiter = "|", header = None)
    df.columns = data['field']
    df = pd.concat([df]*int((count/1000)), ignore_index = True)  
    
    uli_1 = {'uli':list(['BANK1LEIFORTEST12345']*int((count)))}
    df1 = pd.DataFrame(uli_1)
    df1["uli"] = df1.apply(lambda x: x.uli + char_string_gen(23), axis = 1)
    df1['uli'] = df1.apply(lambda x: x.uli + check_digit_gen(ULI = x.uli), axis = 1)
    uli_check(df1)
    
    df['uli'] = df1['uli']
    
    df.to_csv("lar_" + str(count) + ".txt", sep = "|", index = False, header= None)
    
    with open("lar_" + str(count) + ".txt", 'r' ) as f:
        lar_count = f.readlines()
    
    with open("edits_files/ts_row.txt", 'a') as append_lar:
        append_lar.writelines(lar_count)
    
    os.remove("lar_" + str(count) + ".txt")
    return os.rename("edits_files/ts_row.txt", "edits_files/file_bank1_" + str(count) + ".txt")

    
    

In [12]:
large_files_bank1(file=file, count=1000)

Done
